# Entity "C" (Classifier)

Below is code of Entity "C", which is in control of the classifier module:

<img src='../img/sample_05_C.png'>

In [1]:
from tensorflow.python import reduce_mean
from tensorflow.python.ops.nn_ops import softmax_cross_entropy_with_logits
from tensorflow.python.training.adadelta import AdadeltaOptimizer
from tensorflow.python.training.adagrad import AdagradOptimizer
from tensorflow.python.training.optimizer import Optimizer
from dopapy.core.services.instance.ai.tfmodel.ilabeled import ITFLabeledLearningModel
from dopapy.core.services.instance.ai.tfmodel.labeled import LabeledLearningTFModelServiceInstance
from dopapy.utils.common.guid import GUID
from tensorflow.contrib.eager.python.network import Network
from tensorflow.contrib.eager.python.tfe import enable_eager_execution
from tensorflow.python.layers.core import Dense, Dropout, Flatten
from dopapy import *
from dopapy.trading import *
import dopapy.types as dd
import warnings
import tensorflow as tf
from datetime import datetime
from tensorflow.contrib.eager.python import tfe
warnings.filterwarnings('ignore') #Thats just to ignore warnings that are irrelevant for this type of sample
enable_eager_execution()

Definition of network graph as a standard tensorflow Network (using tensorflow eager mode):

In [2]:
class ClassifierNetwork(Network):

    def __init__(self):
        super().__init__()
        self.track_layer(Dense(units=32))
        self.track_layer(Dense(units=10))

    def call(self, inputs, **kwargs):
        result = inputs
        for layer in self.layers:
            result = layer(result)
        return result

Definition of model, which includes in addition to the Network also the chosen optimizer:

In [3]:
class ClassifierModel(ITFLabeledLearningModel):

    def __init__(self):
        super().__init__()
        #self.graph = tfe.IsolateTest()
        #with self.graph.as_default():
        #with self.graph:
        self._network = ClassifierNetwork()
        self._optimizer = AdagradOptimizer(learning_rate=1e-2)

    def get_network(self) -> Network:
        return self._network

    def get_optimizer(self) -> Optimizer:
        return self._optimizer

    def loss_function(self, input_data, labels):
        y_ = self._network.call(input_data)
        loss = softmax_cross_entropy_with_logits(labels=labels, logits=y_)
        loss = reduce_mean(loss)
        return loss

Below we define that every user creates it's own instance:

In [4]:
class ClassifierCreator(IServiceInstanceCreator):
    def create_instance(self, service_instance_id: GUID) -> LabeledLearningTFModelServiceInstance:
        return LabeledLearningTFModelServiceInstance(ClassifierModel(), service_instance_id)

Creating the service and reward descriptors:

In [5]:
init_logging()
in_advance = 100
classifier_session = create_session('password123456', '/data/dopamine/keystore_dir_b/', '127.0.0.1',
                                  int(100E18), int(10E18), web_server_port=8502)
input_descripotr = dd.TensorDescriptor(size=[-1,3872], store_file_prefix="provider_%s" % 2)
output_descriptor = dd.TensorDescriptor(size=[-1,10])
reward_desc = create_reward_descriptor(min_dopamine_price=3)
reward_desc.add_payment(dd.RewardPaymentType.FORWARD, min_price=0, in_advance_quantity=1)
reward_desc.add_payment(dd.RewardPaymentType.RETURN_GRADIENT, min_price=3,
                        in_advance_quantity=in_advance)
reward_desc.add_payment(dd.RewardPaymentType.UPDATE, min_price=3,
                        in_advance_quantity=in_advance)

creator = ClassifierCreator()
service_type = dd.ServiceType.Learning.LABELED
service_desc = dd.ServiceDescriptor(input_descriptors=[input_descripotr],
                                    output_descriptors=[output_descriptor],
                                    service_role=dd.ServiceRole.PROVIDER,
                                    service_type=service_type)
calssifier_services = create_quote_services(
    session=classifier_session,
    qty=1,
    side=SELL,
    service_descriptor=service_desc,
    reward_descriptor=reward_desc,
    service_instance_creator=creator,
    quote_id=5)

And let the service begin...

In [6]:
calssifier_services.publish()